<h1 style="text-align:center;">World of Forumcraft</h1>

In [1]:
import pandas as pd
import MySQLdb as sql

#JSON credentials
login = pd.read_json(os.getcwd() + "/db_credentials.json", typ="series").to_dict()

def connect():
    #db connection
    conn = mysql.connect(host=login["host"],
                         user=login["user"],
                         passwd=login["passwd"],
                         db=login["db"])            
    cur = conn.cursor()
    return conn, cur

def readData(cols):
    conn, cur = connect()
    QUERY = "SELECT %s FROM forum_data" % (",".join(cols))
    data = pd.read_sql(QUERY, con=conn)
    return data

Running data = readData(['datetime']) and converting datetime.date() types into timestamps, I then plotted forum activity over the last seven years.

In [2]:
def timeSeries(dates, counts):
    """Forum activity over the last 7 years."""
    cmap = plt.cm.get_cmap('winter')
    Y,X = np.histogram(timestamps,len(counts),normed=1.0)
    y_span = Y.max()-Y.min()
    colors = [cmap(((y-Y.min())/y_span)) for y in Y]
    y_max = counts.sum()
    
    plt.figure(figsize=(20,5))
    plt.title('US WoW Forums: Last 7 Years')
    plt.ylabel('Forum Activity')
    plt.bar(X[:-1],Y*y_max, width=((X.max()-X.min())/(len(X))), color=colors, align='center',linewidth=0)
    locs = np.arange(X.min(),X.max()+1, ((X.max()-X.min())/7))
    labs = [datetime.date.fromtimestamp(x) for x in locs]
    
    cata_date = time.mktime(datetime.date(2010,12,7).timetuple())
    plt.axvline(x=cata_date,color='red',linestyle='dashed')
    plt.text(cata_date, 0.8,'Cataclysm', color='red', rotation=45)
    
    mists_date = time.mktime(datetime.date(2012,9,25).timetuple())
    plt.axvline(x=mists_date,color='red',linestyle='dashed')
    plt.text(mists_date, 0.8,'Mists of Pandaria', color='red', rotation=45)
    
    warlords_date = time.mktime(datetime.date(2014,11,13).timetuple())
    plt.axvline(x=warlords_date,color='red',linestyle='dashed')
    plt.text(warlords_date, 0.8,'Warlords of Draenor', color='red', rotation=45)
    
    legion_date = time.mktime(datetime.date(2016,8,30).timetuple())
    plt.axvline(x=legion_date,color='red',linestyle='dashed')
    plt.text(legion_date, 0.8,'Legion', color='red', rotation=45)
    
    plt.xticks(locs, labs, ha='right',rotation=45)
    plt.axis('tight')
    plt.xlim([X.min(), X.max()])
    plt.tight_layout()
    plt.show()

<img src="/portfolio/img/wow_plots/WoW_Forum_Activity.png" style="max-width: 900px;">

There's a very noticeable decline in forum activity over the last seven years, which must be a combination of players both leaving the game and players no longer having an interest or need for the forums. Let's break it down even further by year.

In [3]:
def yearPosts(a,b,counts):
    counts = counts
    idx = counts.index
    total = counts[(idx >= datetime.date(a,1,1)) & (idx <= datetime.date(b,1,1))].sum()
    return total

def yearActivity():
    """Total Year Activity"""
    plt.figure()
    plt.title('Forum Activity by Year')
    y = [yearPosts(2000+x,2000+x+1,counts) for x in range(11,18)]
    x = np.arange(0,7,1)
    labels = [str(2000+x) for x in range(11,18)]
    plt.xticks(x,labels)
    plt.bar(x,y, color='green',alpha=0.75)
    plt.ylabel('Posts')
    plt.xlabel('Year')
    plt.show()

<img src="/portfolio/img/wow_plots/WoW_Forum_Activity_By_Year.png" style="max-width: 600px;">

There's a clear drop in forum activity by roughly 37.5% from 2011 to 2016. Could this be due to player's not responding well to recent expansions, e.g. Mists of Pandaria, Warlords of Draenor? To find out, I made sure to leave out duplicate character names so that we could tally up how many unique players were active on the forums per expansion era.

In [4]:
def postByExpansion():
    """Individual player activity."""
    plt.figure()
    plt.title('WoW Forum Posts By Expansion')
    y = df['expansion'].value_counts().values
    x = np.arange(1,5,1)
    labels = ['Cataclysm', 'Mists of Pandaria', 'Warlords of Draenor', 'Legion']
    plt.bar(x,y, color='blue', alpha = 0.5)
    plt.xticks(x,labels, rotation=45)
    plt.ylabel('Player Posts')
    plt.show()

<img src="/portfolio/img/wow_plots/WoW_Forum_Activity_By_Expansion.png" style="max-width: 600px;">

Clearly, the number of unique posters has dwindled since Cataclysm, with a large dip occurring after Mists of Pandaria. Legion is about halfway through its run, and with a new expansion arising every two years or so, and so the actual forum activity by the end of its run should be much higher by then. So if players are leaving the forums, exactly how long are they staying? To find this out, we can determine the time of each player's first and most recent post and calculate the difference. Note that players who posted once will be considered to have used the forum for a single day rather than zero days.

In [5]:
def durations():
    """Average number of days spent active on forums per player."""
    plt.figure()
    x = df['days']
    plt.title('Average Duration of Forum Participation (Bins = 100)')
    plt.hist(x, normed=True, bins=100, color='red')
    plt.axis('tight')
    plt.ylabel('Duration Probability')
    plt.xlabel('Days')
    plt.xlim([x.min()-50,x.max()+50])
    plt.tight_layout()
    plt.show()

<img src="/portfolio/img/wow_plots/WoW_Forum_Durations.png" style="max-width: 600px;">

This is a rather interesting find! Assuming that players post with their primary characters, the WoW community appears somewhat polarized, with two groups having formed: one where player's are active for a single day and the other where players are active for years. This, of course, does not mean such players exhibit continuous activity, but it at least means that these players, or at least their game characters, have returned to the forums rather than have been dissolved into the twisting nether! With this sort of distribution, is it possible to form a decent model which could predict the likelihood of a player/character returning for a second post? For simplicity, we can try setting our labels as One-off Posters and Return Posters with post counts equal to 1 and greater than 1, respectively. 

In [6]:
def readSamples(cols):
    conn, cur = connect()
    QUERY_SINGLE_POSTS = """
                        SELECT %s FROM forum_data AS r1 JOIN
                        (SELECT (RAND() * (SELECT MAX(id) FROM forum_data)) AS id)
                        AS r2 WHERE (r1.id >= r2.id AND postCount = 1) ORDER BY r1.id ASC
                        LIMIT 500000;
                        """ % (",".join(cols))
    
    QUERY_MULTI_POSTS = """
                        SELECT %s FROM forum_data AS r1 JOIN
                        (SELECT (RAND() * (SELECT MAX(id) FROM forum_data)) AS id)
                        AS r2 WHERE (r1.id >= r2.id AND postCount > 1) ORDER BY r1.id ASC
                        LIMIT 500000;
                        """ % (",".join(cols))

    x = pd.read_sql(QUERY_SINGLE_POSTS, con=conn)
    y = pd.read_sql(QUERY_MULTI_POSTS, con=conn)
    data = pd.concat([x, y], axis=0)
    return data

The above code allows us to query a random selection of One-off and Return posters for modeling purposes. Running a model on the entire dataset would not be reasonable for my meek machine.

In [7]:
def sequence():
    model = readSamples(['lvlRaceClass','postCount','postText'])

    """We need to set our labels for one-time posters vs. return posters."""
    model['posCount'] = model['postCount'].replace({r'[a-zA-z]+': np.nan}, regex=True)
    model = model.dropna().reset_index(drop=True)
    model['postCount'] = model['postCount'].astype('int')
    model['poster'] = np.where(model['postCount'] == 1, 'One-off Poster', 'Return Poster')

    """Let's extract level data from the players to add as a feature."""
    Levels = {r".*NaN.*": np.nan,
              r"[a-zA-z ]": ""}
    model['level'] = model['lvlRaceClass'].replace(Levels, regex=True)
    model = model.dropna().reset_index(drop=True)
    model['level'] = model['level'].astype('int')
    model['level'] = (model['level']-model['level'].min())/(model['level'].max()-model['level'].min())

    from sklearn.utils import shuffle
    model = shuffle(model)

The above sequence of commands was used to import the data, set our labels, and clean up the level of each player as a feature. We will also be vectorizing player posts to extract further, more definitive, features.

In [8]:
#Vectorize post text.
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorization_of_posts():
    def my_tokenizer(words):
        lem = WordNetLemmatizer()
        return [lem.lemmatize(word,'n').lower() for
                word in nltk.word_tokenize(words) if len(word.lower()) >= 3]

    vectorizer = TfidfVectorizer(ngram_range=(1,3), stop_words='english', tokenizer=my_tokenizer)
    vector_matrix = vectorizer.fit_transform(model['postText'])

from scipy.sparse import hstack
def features_and_labels():
    features = hstack((vector_matrix, np.array(model['level'])[:, None]))
    labels = model['poster']

The above two functions are then used to vectorize each post text using a customized tokenizer. Once the sparse Tfidf matrix is completed, we stack player level column with the matrix using hstack() so that the model can be trained on a combination of player level and post text. Labels are, of course, the poster's status of either a One-off Poster or a Return Poster.

In [9]:
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

def grid_search():
    f_train, f_test, l_train, l_test = train_test_split(features, labels, test_size = 0.3)

    for score in ['precision','recall']:
    
        parameters = [{'class_weight':['balanced'],'C':[1.0, 10.0, 25.0, 50.0, 75.0, 100.0]}]
    
        print("### Tuning hyper-parameters for %s ###" % score)
        print()

        clf = GridSearchCV(LogisticRegression(), parameters, n_jobs=4, 
                           cv = 5, scoring='%s_macro' % score)
        clf.fit(f_train, l_train)

        print("Best parameter set found:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
    
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
        
        print()

        print("Classification report: ")
        print()
        print("The model is trained on the full development set.")
        print("The scores are computed on the full evaluation set.")
        print()
        l_true, l_pred = l_test, clf.predict(f_test)
        print(classification_report(l_true, l_pred))
        print()

<img src="/portfolio/img/wow_plots/classification_wow_posters.png" style="max-width: 600px;">

Performing a grid search can help with the tuning hyperparameters for our classifier. With a non-exhaustive search we're able to find a decent value for 'C' to improve both precision and recall on our test set. The grid search can be very slow, and is especially slow on my meek machine. With a bit of tuning and patience we could improve this 91% precision/recall score even further. Our options would be to either extract further features or try different classifiers. For now, we have a good launching point for predicting the probability that someone will be a one-off poster or a return poster on the forums based primarily on the text of their post and character's level.

<h1 style="text-align: center;">Recommendation System</h1>
<br>
<br>
Aside from predicting one-time posters versus return posters, we may also want to use the available features in the data to build a recommendation system. In this case, it is desirable to use each player's level, achievements, and faction since we want to recommend players of on the same side with similar skill levels. 

In [10]:
import numpy as np

def readSample(cols):
    conn, cur = connect()
    QUERY = "SELECT %s FROM forum_data ORDER BY RAND() LIMIT 1000000" % (",".join(cols))
    QUERY = """
            SELECT %s FROM forum_data AS r1 JOIN
            (SELECT (RAND() * (SELECT MAX(id) FROM forum_data)) AS id)
            AS r2 WHERE r1.id >= r2.id ORDER BY r1.id ASC
            LIMIT 3000000;
            """ % (",".join(cols))
    data = pd.read_sql(QUERY, con=conn)
    return data

Factions = {r'.*Human.*': 'Alliance',
            r'.*Dwarf.*': 'Alliance',
            r'.*Gnome.*': 'Alliance',
            r'.*Night Elf.*': 'Alliance',
            r'.*Draenei.*': 'Alliance',
            r'.*Worgen.*': 'Alliance',
            r'.*Orc.*': 'Horde',
            r'.*Troll.*': 'Horde',
            r'.*Undead.*': 'Horde',
            r'.*Tauren.*': 'Horde',
            r'.*Goblin.*': 'Horde',
            r'.*Blood Elf.*': 'Horde',
            r'.*Pandaren.*': 'Pandaren',
            r'.*NaN.*': 'Staff'}

Levels = {r".*NaN.*": np.nan,
          r"[a-zA-z ]": ""}

Achievements = {r".*NaN.*": np.nan}

data = readSample(['lvlRaceClass','achievements'])

data['faction'] = data['lvlRaceClass'].replace(Factions, regex=True)
data['level'] = data['lvlRaceClass'].replace(Levels, regex=True)
data['achievements'] = data['achievements'].replace(Achievements, regex=True)

df = data[['level','achievements','faction','lvlRaceClass']].copy()
df = df.dropna().reset_index(drop=True)
df['faction'] = pd.get_dummies(df['faction'])
df = df.drop_duplicates().reset_index(drop=True)
player_arrays = df[['level','achievements','faction']].as_matrix().astype('float')

Here, we are just setting and cleaning up feature columns. The important bit of code is when we set these features into a matrix. In order to recommend matched players we'll be using cosine distance which essentially quantifies how close two vectors are to one another. 

In [11]:
class wowRecommender():

    def __init__(self):
        self.player_arrays = player_arrays
    
    def cosDist(self, x,y):
        num = np.dot(x,y)
        den = np.sqrt(np.dot(x,x)*np.dot(y,y))+1e-6
        return (1.-(num/den))

    def cosSim(self, player, player_arrays):
        N = len(player_arrays)
        distances = np.zeros(N)
        for idx in range(N):
            distances[idx] = self.cosDist(player, self.player_arrays[idx])
        return distances
            
    def recommend(self, idx):
        player = self.player_arrays[idx]
        print("\nPlayer: "+str(df['lvlRaceClass'][idx])+str(" ")+str(df['achievements'][idx]))
        distances = self.cosSim(player, self.player_arrays)
        ids = distances.argsort()[1:6]
        print("\nRecommendations: ")
        for i in range(len(ids)):
            print(str(i+1) + ". " + str(df['lvlRaceClass'][ids[i]])+str(" ")+str(df['achievements'][ids[i]]))

There are three primary methods in the recommender class above. The first, cosDist, calculates the cosine distance between two vectors by returning 1 - cos(theta), where cos(theta) is represented by the scalar product term and theta is the angle between the two vectors, x and y. In the absence of negative values, the cosine term varies between 0 and 1 and so cosine distances of 0 represent the case where vector x and y are identical. The second method, cosSim, calculates an array of cosine distances from our feature matrix where there are N*(N-1)/2 unique pairs between all vector rows. Lastly, the recommend method sorts the 5 shortest cosine distances from the player in question. Let's see it in action!

In [12]:
recommender = wowRecommender()
recommender.recommend(81132)


Player: 100 Blood Elf Monk 10625

Recommendations: 
1. 100 Pandaren Warrior 10625
2. 100 Blood Elf Monk 10625
3. 100 Undead Priest 10625
4. 100 Tauren Druid 10625
5. 100 Blood Elf Rogue 10625


In [13]:
recommender.recommend(44020)


Player: 85 Draenei Death Knight 2870

Recommendations: 
1. 85 Human Mage 2870
2. 85 Night Elf Rogue 2870
3. 85 Worgen Rogue 2870
4. 85 Draenei Death Knight 2870
5. 85 Human Priest 2870


In [14]:
recommender.recommend(331)


Player: 110 Blood Elf Death Knight 11305

Recommendations: 
1. 110 Tauren Paladin 11305
2. 110 Blood Elf Paladin 11305
3. 110 Orc Rogue 11305
4. 110 Blood Elf Hunter 11305
5. 110 Undead Priest 11305


To our delight, the system recommends other players on the same faction who have similar level and achievement scores. Given the practicality of playing with others with different skill sets, there's no need to include in-game class. This kind of system can be fully generalized and extended to all sorts of other datasets, allowing us to make recommendations elsewhere.

<h1 style="text-align: center;">Conclusions<h1>

<ul>
<li>WoW forum activity has been significantly declining over the past 7 years.</li>
<li>A large number of players/characters have only used the forums once.</li>
<li>A classifier can be trained to predict whether a player will be a one-time poster or not.</li>
<li>A Recommendation system can be implemented to match like-leveled players with similar achievement scores.</li>
</ul>